# Initial Model Training Change log

1. Changed l=40 to l=10
2. Increasing Batch sixed from 128 to 512 to increase execution code
3. Reducing no of epochs; also seems l=40 marked in hyperparameters is not effecting so changing l value in model building to 4
4. putting compression to 1.0 to make model wider

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

import time
import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline
np.random.seed(2017) 

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 512 #128
num_classes = 10
epochs = 50
l = 10 # reducing layers from 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoding 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [9]:
#model building 
num_filter = 12
dropout_rate = 0.2
l = 4 #12
compression=1.0

input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
#First 20 epochs
epochs=20
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

In [0]:
#Training for another 20 epochs
epochs=20
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

In [0]:
from google.colab import files

files.download('DNST_model.h5')

# Simple TFRecords

Following: https://medium.com/@moritzkrger/speeding-up-keras-with-tfrecord-datasets-5464f9836c36


In [0]:
# Helperfunctions to make your feature definition more readable
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


In [0]:
#importing libs for next steps
import sys
import os
from matplotlib.image import imread
from keras.utils import np_utils

In [12]:
# load data in numpy
(train_features, train_labels), (test_features, test_labels) = cifar10.load_data()

num_train, img_channels, img_rows, img_cols =  train_features.shape
num_test, _, _, _ =  test_features.shape
num_classes = len(np.unique(train_labels))

#images = images.astype('float32')/255
#labels = np_utils.to_categorical(labels, num_classes)
'''
train_features = train_features.astype('float32')/255
test_features = test_features.astype('float32')/255
# convert class labels to binary class labels
#print (train_labels.shape)
train_labels = np_utils.to_categorical(train_labels, num_classes)
#print (train_labels.shape)
test_labels = np_utils.to_categorical(test_labels, num_classes)
'''

"\ntrain_features = train_features.astype('float32')/255\ntest_features = test_features.astype('float32')/255\n# convert class labels to binary class labels\n#print (train_labels.shape)\ntrain_labels = np_utils.to_categorical(train_labels, num_classes)\n#print (train_labels.shape)\ntest_labels = np_utils.to_categorical(test_labels, num_classes)\n"

In [0]:
#defining file path to store train tfrecords
data_dir = '/content/'
filename = "train.tfrecords"

FILEPATH = os.path.join(data_dir, filename)

# Open a TFRecordWriter for to create train tfrecords.
with tf.python_io.TFRecordWriter(FILEPATH) as writer:
  
  for i in range(num_train):
    # Define the features of your tfrecord
    feature = {'image':  _bytes_feature(tf.compat.as_bytes(train_features[i].tostring())),
               'label':  _int64_feature(int(train_labels[i]))}
    
    # Serialize to string and write to file
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(example.SerializeToString())
      

In [0]:
#defining file path to store test tfrecords
data_dir = '/content/'
filename = "test.tfrecords"

FILEPATH = os.path.join(data_dir, filename)

# Open a TFRecordWriter for to create test tfrecords.

with tf.python_io.TFRecordWriter(FILEPATH) as writer:
  
  for i in range(num_test):
    # Define the features of your tfrecord
    feature = {'image':  _bytes_feature(tf.compat.as_bytes(test_features[i].tostring())),
               'label':  _int64_feature(int(test_labels[i]))}
    
    # Serialize to string and write to file
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(example.SerializeToString())

In [0]:
def _parse_function(proto):
    # define your tfrecord again. Remember that you saved your image as a string.
    keys_to_features = {'image': tf.FixedLenFeature([], tf.string),
                        "label": tf.FixedLenFeature([], tf.int64)}
    
    # Load one example
    parsed_features = tf.parse_single_example(proto, keys_to_features)
    
    # Turn your saved image string into an array
    parsed_features['image'] = tf.decode_raw(
        parsed_features['image'], tf.uint8)
    
    return parsed_features['image'], parsed_features["label"]

In [16]:
#defining file path to load train tfrecords
data_dir = '/content/'
filename = "train.tfrecords"

FILEPATH = os.path.join(data_dir, filename)

dataset = tf.data.TFRecordDataset(FILEPATH)

# Maps the parser on every filepath in the array. You can set the number of parallel loaders here
dataset = dataset.map(_parse_function, num_parallel_calls=8)

dataset = dataset.repeat()

dataset = dataset.shuffle(2)

dataset = dataset.batch(batch_size)

#Creating iterator
iterator = dataset.make_one_shot_iterator()

#create tf rep for iterator
image, label = iterator.get_next()

#normalize images
image = tf.reshape(image, [-1, 256, 256, 1] )
image = tf.to_float(image) / 255.0

# Create a one hot array for your labels
label = tf.one_hot(label, num_classes)


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
Instructions for updating:
Use `tf.cast` instead.


In [0]:
image.get_shape
label.get_shape
label.value_index

0

In [0]:
#model building 
num_filter = 12
dropout_rate = 0.2
l = 4 #12
compression=1.0

input = Input(tensor=image)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [18]:
train_model = Model(inputs=[input], outputs=[output])
train_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 256, 256, 12) 108         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, 256, 256, 12) 48          conv2d_21[0][0]                  
__________________________________________________________________________________________________
activation_21 (Activation)      (None, 256, 256, 12) 0           batch_normalization_21[0][0]     
____________________________________________________________________________________________

In [19]:
# Set Loss function and Optimizer
'''
train_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              target_tensors=[label]) #metrics=['accuracy'],
'''
train_model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001),
                    loss='mean_squared_error',
                    metrics=['accuracy'],
                    target_tensors=[label])

In [22]:
#Training with tfrecords data
#First 20 epochs
epochs=20
STEPS_PER_EPOCH = int(num_train // (batch_size/256 * batch_size/256 * 3 *2))
train_model.fit(image, label,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

#fit(epochs=epochs, steps_per_epoch=STEPS_PER_EPOCH)


Epoch 1/20


UnboundLocalError: ignored

In [0]:
print (batch_size/256 * batch_size/256 * 3)
print (num_train // (batch_size/256 * batch_size/256 * 3))

12.0
4166.0


# Following tech on "What's new in tensorflow 2.0"
shared by Aditya